<a href="https://colab.research.google.com/github/Mr-McGL/RExamsUtils/blob/dev%2Fmgarcia/pyexams/test/PyExamsUtilsTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyExamsUtils: Build and test

## Init extensions

In [ ]:
%%capture __script_and_ext__output__
!mkdir -p /content/scripts/
%pushd /content/scripts/
try:
  !wget -O importExt_script.py https://gist.githubusercontent.com/Mr-McGL/661ae7e50a1cfe7dfd5c0b23216bf0c3/raw/importExt_script.py

  %cd ..
  %run scripts/importExt_script skip_ext
  %run scripts/importExt_script pNpDir_ext
  %run scripts/importExt_script writefileE_ext
  %run scripts/importExt_script resize_output_ext
  %run scripts/importExt_script repo_ext

finally:
  %popd

In [ ]:
%%resize_output -hh 100px
__script_and_ext__output__()

## Build
To set up the build, make sure to configure the following parameters:

* `url`: The repository URL.
* `branch`: Specify the branch you want to build from.

In [ ]:
url = "https://github.com/Mr-McGL/RExamsUtils.git"
branch = "dev/mgarcia"

In [ ]:
%%resize_output -hh 400px -o __build_output__
%%pushAndPopDir -swd
#private repositories
#%%repo -t github -f reu  {url} {tokenFileID}
#%%pushAndPopDir -swd

import importlib

def prntStep(*args, **kwargs):
  print(f"\x1b[1m\x1b[36m", end='')
  print(*args, "\x1b[0m", **kwargs)

def installPKG(pkg):
  if importlib.util.find_spec(pkg) is None:
    !pip install {pkg}
  else:
    !pip install --upgrade {pkg}

################################
# Required Packages
prntStep("\nInstalling Required Packages")
installPKG("build")
installPKG("virtualenv")

##############################
# Clonning the Repository
# (for private repositories use %%repo)
prntStep("\nCloning the repository")
!git clone --recursive {url} reu

##############################
# Set Branch
prntStep("\nSetting the required branch")
%cd reu
!git checkout {branch}


##############################
# Build
prntStep("\nBuilding package")
%cd pyexams
#!python3 -m build
!python3 -m build

##############################
# Copying Package
prntStep("\nCopying package files")
!mkdir -p ../../pkgs
!cp -rf ./dist/* ../../pkgs

##############################
# Delete repository
# (for private repositories use %%repo)
prntStep("\nDeleting repository")
%cd ../..
!rm -r reu

In [ ]:
%%resize_output -hh 200px
__build_output__

In [ ]:
%%skip {True}
%%pushAndPopDir -swd
!rm -r reu

## Upload Binaries

Before uploading the package binaries, ensure that you've set the following parameters:

* `url`: The URL of the repository for binaries.
* `tokenFileID`: The Google Drive file ID where the your tokens are stored.
* `name`: Your user name.
* `email`: Your email address.

***How to create a token file:***

<details>

```py
%%resize_output -hh 60px
!apt-get install xattr
```

```py
%%pushAndPopDir -swd

tokenFileName = "_tokens_.json"
folderName =""

from google.colab import drive, userdata
from subprocess import getoutput
import json

drive.mount('gdrive')
repoURL = userdata.get('repoURL')

%pushd gdrive/MyDrive/{folderName}

with open(tokenFileName,"w") as f:
  f.write(json.dumps({
  "github": {
		  "user": userdata.get('user_name'),# #<---- Your user name
		  "email": userdata.get('user_email'), #<---- Your email
		  "token": userdata.get('token') #<---- Your token
	  }
  }))

%popd

drive.flush_and_unmount()
```

```py
#https://gist.github.com/korakot/9bec3d134a70aa9797e17bc0adeb6663
drive.mount('gdrive')
%pushd gdrive/MyDrive/{folderName}
file_ID = getoutput(f"xattr -p 'user.drive.id' {tokenFileName}")
%popd
drive.flush_and_unmount()
print(file_ID)
```

</details>

In [ ]:
# @title  { run: "auto", form-width: "50%", display-mode: "form" }
skip_upload = True # @param {type:"boolean"}

In [ ]:
%%skip {skip_upload}

#tokenFileID = "1cCDt2uhK4vnwpiStmyyjzci3u90JaIE3" #safe
#Stored as Google Colab secret
from google.colab import userdata
tokenFileID = userdata.get('tokenFileID')
email = userdata.get('user_email')
name = userdata.get('user_name')
url = "https://github.com/Mr-McGL/RExamsUtilsBinaries.git"

In [ ]:
%%skip {skip_upload}
!git config --global user.email {email}
!git config --global user.name {name}

In [ ]:
%%resize_output -hh 100px -o __upload_output__
%%skip {skip_upload}
%%pushAndPopDir -swd
%%repo -t github -f reu_bin  {url} {tokenFileID}
%%pushAndPopDir -swd

%cd reu_bin

!mkdir -p ./pkgs
!cp -rf ../pkgs/* ./pkgs
!git add *
!git commit -a -m "PKGs updated"
!git push

In [ ]:
__upload_output__

## Install

In [ ]:
%%resize_output -hh 400px -o __build_install__
%%pushAndPopDir -swd pkgs
import os
#files = [entry.name for entry in os.scandir('.') if entry.is_file() and entry.name.endswith(".whl")]
files = sorted([f for f in os.listdir('.') if f.endswith(".whl")])
!pip install --force-reinstall {files[-1]}

In [ ]:
%%resize_output -hh 200px
__build_install__

In [ ]:
%%skip True
%%resize_output -hh 200px
%%pushAndPopDir -swd pkgs

files = sorted([f for f in os.listdir('.') if f.endswith(".whl")])
!pip uninstall {files[-1]}

In [ ]:
import pyexams.fprnt as exfprint
exfprint.fprnt("Hola", fg="r")

import pyexams.ex
pyexams.ex.run("ls")

import pyexams
pyexams.types.Struct({1:2})

## Test

### Development

In [ ]:
# @title  { run: "auto", display-mode: "form" }
dev = False # @param {type:"boolean"}

In [ ]:
#Init
%%skip {not dev}
%%resize_output -hh 100px
%%pushAndPopDir -swd

url = "https://github.com/Mr-McGL/RExamsUtils.git"
branch = "dev/mgarcia"

import importlib

def prntStep(*args, **kwargs):
  print(f"\x1b[1m\x1b[36m", end='')
  print(*args, "\x1b[0m", **kwargs)

def installPKG(pkg):
  if importlib.util.find_spec(pkg) is None:
    !pip install {pkg}
  else:
    !pip install --upgrade {pkg}

prntStep("\nInstalling Required Packages")
installPKG("build")
installPKG("virtualenv")

prntStep("\nCloning the repository")
!git clone --recursive {url} reu

prntStep("\nSetting the required branch")
%cd reu
!git checkout {branch}

In [ ]:
#Build and install
%%skip {not dev}
%%resize_output -hh 100px
%%pushAndPopDir -swd reu/pyexams

import os

#prntStep("\nUninstalling previous version")
#if os.path.exists("./dist"):
#  %pushd "./dist"
#  files = sorted([f for f in os.listdir('.') if f.endswith(".whl")])
#  !pip uninstall -y {files[-1]}
#  %popd

prntStep("\nCleaning repo")
dzi = !find . -name *Zone.Identifier
dcch = !find . -name __pycache__
degg = !find . -name *.egg-info

for f in dzi + dcch + degg:
  !rm -r {f}

if os.path.exists("./build"):
  !rm -r ./build

if os.path.exists("./dist"):
  !rm -r ./dist


prntStep("\nBuilding repo")
!python3 -m build

prntStep("\nInstalling repo")
#!pip uninstall .
%cd "./dist"
files = sorted([f for f in os.listdir('.') if f.endswith(".whl")])
!pip install --force-reinstall {files[-1]}
#!pip install --force-reinstall .

In [ ]:
%%skip {not dev}

import pyexams.fprnt as exfprint
exfprint.fprnt("Hola", fg="r")

import pyexams.ex
pyexams.ex.run("ls")

import pyexams
pyexams.types.Struct({1:2})

In [ ]:
%%skip {not dev}
%%pushAndPopDir -swd
prntStep("\nDeleting repository")
!rm -r reu

### ***fprnt*** - Formatted print tests

Los siguientes diccionarios incluyen caracteres especiales.

In [ ]:
from pyexams.fprnt import (format as frmt, fgColor as fgc,
                             bgColor as bgc, special as spcl)

print(f"{frmt.bold+frmt['italic']}Bold and italic{frmt.resetBold} - italic{frmt.reset} - default")
print(f"{frmt.b+frmt.i}Bold and italic{frmt.ri} - italic{frmt.rst} - default")

print("")
print(f"{fgc.cyan+bgc.magenta}Cyan and magenta{frmt.rst} | default")
print(f"{fgc.c+bgc.m}Cyan and magenta{fgc.default} | default and Magenta {bgc.d} default")

print(f"\n Unseen {spcl.lbeging}Line deleted")


In [ ]:
from pyexams.fprnt import fgvColor as fgvc, bgvColor as bgvc, fgRGB, bgRGB


print(f"{fgvc(1)+bgvc(50)}Test 1")
print(f"{fgRGB(200,0,0)+bgRGB(200,200,200)}Test 2")

print(f"{fgRGB(200,0,0)+bgRGB(200,200,200)}Test 2")



In [ ]:
from pyexams.fprnt import setFormat as _sf

print(f"{_sf(('fg',200,0,0), ('bg',200,200,200),'b' )}Test 1")
print(f"{_sf(('fg',200), ('bg',1),'b' )}Test 2")
print(f"{_sf(('fg','r'), ('bg','w'),'b' )}Test 3")
print(f"{_sf('fgc','bgw','b' )}Test 4")
print(f"{_sf('fgred','bgw','b' )}Test{_sf('r')} 5")

La siguiente funcion permite seleccionar un formato.

In [ ]:
from pyexams.fprnt import fprnt, fprntNNL

fprnt("Test 1", fg = "r", bg = "w", frmt = "u")
fprnt("Test 2", fg = 2, bg = 3)
fprnt("Test 3", fg = (0,0,200), bg = (100,0,0))

fprntNNL("Test", fg = (0,100,200), bg = (100,0,0))
fprnt(" 4", fg = (200,50,0), bg = (0,200,0))

fprnt("Test", fg = (0,100,200), bg = (100,0,0), end=' ')
fprnt("5", fg = (200,50,0), bg = (0,200,0))